<a href="https://colab.research.google.com/github/lyco-p/Phos/blob/main/Phos_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phos. Computational Optical Imaging

**Phos** is a film simulation tool based on "Computational optical imaging". By calculating the transmission of light through film emulsion layers and simulating optical diffusion (bloom) and grain, it recreates the aesthetic of analog film.

### Features
- **GPU Acceleration**: Uses NVIDIA GPU (via CuPy) for high-speed processing.
- **Film Presets**: NC200 (Color), AS100 (B&W), FS200 (High Contrast B&W).
- **Customizable**: Adjust grain, tone mapping, and more.
- **Video Support**: Process videos with frame-by-frame rendering.


In [ ]:
#@title 1. Install Dependencies
#@markdown Installs `cupy-cuda12x` for GPU acceleration and `opencv-python-headless`.

!pip install cupy-cuda12x opencv-python-headless tqdm

import cv2
import numpy as np
import os
import sys
import time
import shutil
import subprocess
from google.colab import files
from IPython.display import Image, display
from tqdm.notebook import tqdm

print("Dependencies installed.")

In [ ]:
#@title 2. Setup Core Library
#@markdown Creates the `core` package structure in the Colab environment.

import os
os.makedirs("core", exist_ok=True)

# Create __init__.py
with open("core/__init__.py", "w") as f:
    f.write("")


In [ ]:
%%writefile core/backend.py
import numpy as np
import cv2
import sys

# Backend selection: CuPy > PyTorch (MPS/CUDA) > NumPy

HAS_GPU = False
BACKEND_TYPE = "CPU"
cp = None
torch = None

# 1. Try CuPy (best for NVIDIA)
try:
    import cupy as cp
    import cupyx.scipy.ndimage
    HAS_GPU = True
    BACKEND_TYPE = "CUDA (CuPy)"
except ImportError:
    pass

# 2. Try PyTorch (best for Mac/MPS, or if CuPy missing on NVIDIA)
if not HAS_GPU:
    try:
        import torch
        import torchvision
        if torch.backends.mps.is_available():
            HAS_GPU = True
            BACKEND_TYPE = "MPS (PyTorch)"
        elif torch.cuda.is_available():
            HAS_GPU = True
            BACKEND_TYPE = "CUDA (PyTorch)"
    except ImportError:
        pass

# -----------------------------------------------------------------------------
# Adapter for PyTorch to mimic NumPy/CuPy API
# -----------------------------------------------------------------------------
class TorchBackendAdapter:
    def __init__(self, device):
        self.device = device
        self.float32 = torch.float32
        self.uint8 = torch.uint8
        
    def asarray(self, arr):
        if isinstance(arr, torch.Tensor):
            return arr.to(self.device)
        return torch.from_numpy(np.array(arr)).to(self.device)
        
    def asnumpy(self, arr):
        if isinstance(arr, torch.Tensor):
            return arr.cpu().numpy()
        return np.array(arr)
        
    def mean(self, arr, axis=None):
        return torch.mean(arr, dim=axis)
        
    def clip(self, arr, min_val, max_val):
        if not isinstance(arr, torch.Tensor):
            # Handle scalar input (mimic numpy.clip behavior for scalars)
            return min(max(arr, min_val), max_val)
        return torch.clamp(arr, min_val, max_val)
        
    def abs(self, arr):
        return torch.abs(arr)
        
    def power(self, arr, exponent):
        return torch.pow(arr, exponent)
    
    def maximum(self, a, b):
        # b can be scalar
        if isinstance(b, (int, float)):
            b = torch.tensor(b, device=self.device, dtype=a.dtype)
        return torch.maximum(a, b)
        
    def dstack(self, arrays):
        return torch.dstack(arrays)
        
    # Nested Random module
    class RandomAdapter:
        def __init__(self, device):
            self.device = device
            
        def normal(self, loc, scale, size):
            # size is a tuple
            return torch.normal(mean=loc, std=scale, size=size, device=self.device)
            
        def choice(self, a, size):
            # simplified choice for [-1, 1] case usage in Phos
            # xp.random.choice([-1, 1], shape)
            if isinstance(a, list) and set(a) == {-1, 1}:
                # Generate 0s and 1s, map to -1 and 1
                # 0 -> -1, 1 -> 1
                rand_bool = torch.randint(0, 2, size, device=self.device)
                return rand_bool * 2 - 1
            else:
                # Fallback implementation for generic choice if needed
                indices = torch.randint(0, len(a), size, device=self.device)
                if isinstance(a, list):
                    a_tensor = torch.tensor(a, device=self.device)
                return a_tensor[indices]

    @property
    def random(self):
        return self.RandomAdapter(self.device)

    def astype(self, arr, dtype):
        return arr.to(dtype)

# -----------------------------------------------------------------------------
# Initialize 'xp'
# -----------------------------------------------------------------------------
if BACKEND_TYPE.startswith("CUDA (CuPy)"):
    xp = cp
    # Add loose function alias if not present (CuPy mirrors NumPy, so xp.array works, but xp.astype(arr) isn't standard numpy function, it's method)
    # We need a unified function.
    
    def astype(arr, dtype):
        return arr.astype(dtype)
        
    xp.astype = astype # Monkey patch or just export it? 
    # processing.py imports xp. We can add it to xp namespace effectively if we wrap it, but standard module doesn't allow easy assign.
    # We should export `astype` from backend.py and use it.

    def to_cpu(arr):
        if isinstance(arr, cp.ndarray):
            return cp.asnumpy(arr)
        return arr

    def to_gpu(arr):
        if isinstance(arr, np.ndarray):
            return cp.asarray(arr)
        return arr

elif "PyTorch" in BACKEND_TYPE:
    if "MPS" in BACKEND_TYPE:
        device = torch.device("mps")
    else:
        device = torch.device("cuda")
        
    xp = TorchBackendAdapter(device)
    # Adapter already has astype method, so xp.astype works!
    
    def to_cpu(arr):
        if isinstance(arr, torch.Tensor):
            return arr.cpu().numpy()
        return arr

    def to_gpu(arr):
        if isinstance(arr, np.ndarray):
            return torch.from_numpy(arr).to(device)
        return arr

else:
    xp = np
    
    # NumPy doesn't have np.astype(arr, dtype).
    def astype_numpy(arr, dtype):
        return arr.astype(dtype)
    xp.astype = astype_numpy
    
    def to_cpu(arr):
        return arr

    def to_gpu(arr):
        return arr



def get_backend_name():
    return BACKEND_TYPE


def gaussian_blur(arr, ksize, sigma):
    """
    Apply Gaussian blur.
    ksize: tuple (k_h, k_w).
    sigma: float.
    """
    if BACKEND_TYPE.startswith("CUDA (CuPy)") and isinstance(arr, cp.ndarray):
        if sigma <= 0:
            k = ksize[0]
            sigma = 0.3 * ((k - 1) * 0.5 - 1) + 0.8
        return cupyx.scipy.ndimage.gaussian_filter(arr, sigma=sigma)
        
    elif "PyTorch" in BACKEND_TYPE and isinstance(arr, torch.Tensor):
        import torchvision.transforms.functional as F
        
        if sigma <= 0:
            k = ksize[0]
            sigma = 0.3 * ((k - 1) * 0.5 - 1) + 0.8
            
        # F.gaussian_blur expects (C, H, W). Arr is (H, W) or (H, W, C)?
        # Phos logic: Single channel lux map -> (H, W).
        # We need to unsqueeze to (1, H, W) or (1, 1, H, W).
        # GaussianBlur supports (..., H, W)
        
        # Ensure ksize is odd
        k = int(ksize[0])
        if k % 2 == 0: k += 1
        kernel_size = [k, k] # (h, w)
        
        # If float64, convert to float32 for MPS (MPS doesn't support float64)
        if arr.dtype == torch.float64:
            arr = arr.to(torch.float32)
            
        if arr.ndim == 2:
             # (H, W) -> unsqueeze -> (1, H, W)
             img_tensor = arr.unsqueeze(0)
             blurred = F.gaussian_blur(img_tensor, kernel_size, [sigma, sigma])
             return blurred.squeeze(0)
        else:
            # (H, W, C) -> Permute -> (C, H, W)
            img_tensor = arr.permute(2, 0, 1)
            blurred = F.gaussian_blur(img_tensor, kernel_size, [sigma, sigma])
            return blurred.permute(1, 2, 0)

    else:
        # NumPy/OpenCV
        return cv2.GaussianBlur(arr, ksize, sigma)

def resize(arr, size, interpolation=cv2.INTER_LINEAR):
    """
    Resize image.
    size: (width, height)
    """
    target_w, target_h = size
    
    if BACKEND_TYPE.startswith("CUDA (CuPy)") and isinstance(arr, cp.ndarray):
        h, w = arr.shape[:2]
        zoom_h = target_h / h
        zoom_w = target_w / w
        if arr.ndim == 2:
            zoom_factors = (zoom_h, zoom_w)
        else:
            zoom_factors = (zoom_h, zoom_w, 1)
        
        order = 1
        if interpolation == cv2.INTER_NEAREST: order = 0
        elif interpolation in [cv2.INTER_CUBIC, cv2.INTER_LANCZOS4]: order = 3
        
        return cupyx.scipy.ndimage.zoom(arr, zoom_factors, order=order)
        
    elif "PyTorch" in BACKEND_TYPE and isinstance(arr, torch.Tensor):
        import torch.nn.functional as F
        
        # F.interpolate expects (N, C, H, W)
        # arr is (H, W) or (H, W, C)
        
        if arr.ndim == 2:
            # (H, W) -> (1, 1, H, W)
            x = arr.unsqueeze(0).unsqueeze(0)
        elif arr.ndim == 3:
            # (H, W, C) -> (1, C, H, W)
            x = arr.permute(2, 0, 1).unsqueeze(0)
            
        mode = 'bilinear'
        if interpolation == cv2.INTER_NEAREST: mode = 'nearest'
        elif interpolation in [cv2.INTER_CUBIC, cv2.INTER_LANCZOS4]: mode = 'bicubic'
        
        align_corners = False 
        # For nearest, align_corners must be None
        if mode == 'nearest': align_corners = None
        
        # Cast to float32 for interpolation (some MPS ops don't support uint8)
        original_dtype = arr.dtype
        x = x.to(torch.float32)
        
        out = F.interpolate(x, size=(target_h, target_w), mode=mode, align_corners=align_corners)
        
        # Cast back if necessary (e.g. for image display)
        # But wait, core logic usually expects float (0-1) for processing, but standardize takes uint8 inputs?
        # standardize takes whatever comes in.
        # If output needs to be uint8, cast back.
        if original_dtype == torch.uint8:
             out = out.clamp(0, 255).to(torch.uint8)
        
        if arr.ndim == 2:
            return out.squeeze(0).squeeze(0)
        elif arr.ndim == 3:
            return out.squeeze(0).permute(1, 2, 0)

    else:
        return cv2.resize(arr, size, interpolation=interpolation)


In [ ]:
%%writefile core/presets.py
def film_choose(film_type):
    if film_type == ("NC200"):
        r_r = 0.77 #红色感光层吸收的红光
        r_g = 0.12 #红色感光层吸收的绿光
        r_b = 0.18 #红色感光层吸收的蓝光
        g_r = 0.08 #绿色感光层吸收的红光
        g_g = 0.85 #绿色感光层吸收的绿光
        g_b = 0.23 #绿色感光层吸收的蓝光
        b_r = 0.08 #蓝色感光层吸收的红光
        b_g = 0.09 #蓝色感光层吸收的绿光
        b_b = 0.92 #蓝色感光层吸收的蓝光
        t_r = 0.25 #全色感光层吸收的红光
        t_g = 0.35 #全色感光层吸收的绿光
        t_b = 0.35 #全色感光层吸收的蓝光
        color_type = ("color") #色彩类型
        sens_factor = 1.20 #高光敏感系数
        d_r = 1.48 #红色感光层接受的散射光
        l_r = 0.95 #红色感光层接受的直射光
        x_r = 1.18 #红色感光层的响应系数
        n_r = 0.18 #红色感光层的颗粒度
        d_g = 1.02 #绿色感光层接受的散射光
        l_g = 0.80 #绿色感光层接受的直射光
        x_g = 1.02 #绿色感光层的响应系数
        n_g = 0.18 #绿色感光层的颗粒度
        d_b = 1.02 #蓝色感光层接受的散射光
        l_b = 0.88 #蓝色感光层接受的直射光
        x_b = 0.78 #蓝色感光层的响应系数
        n_b = 0.18 #蓝色感光层的颗粒度
        d_l = None #全色感光层接受的散射光
        l_l = None #全色感光层接受的直射光
        x_l = None #全色感光层的响应系数
        n_l = 0.08 #全色感光层的颗粒度
        gamma = 2.05
        A = 0.15 #肩部强度
        B = 0.50 #线性段强度
        C = 0.10 #线性段平整度
        D = 0.20 #趾部强度
        E = 0.02 #趾部硬度
        F = 0.30 #趾部软度
    elif film_type == ("FS200"):
        r_r = 0 #红色感光层吸收的红光
        r_g = 0 #红色感光层吸收的绿光
        r_b = 0 #红色感光层吸收的蓝光
        g_r = 0 #绿色感光层吸收的红光
        g_g = 0 #绿色感光层吸收的绿光
        g_b = 0 #绿色感光层吸收的蓝光
        b_r = 0 #蓝色感光层吸收的红光
        b_g = 0 #蓝色感光层吸收的绿光
        b_b = 0 #蓝色感光层吸收的蓝光
        t_r = 0.15 #全色感光层吸收的红光
        t_g = 0.35 #全色感光层吸收的绿光
        t_b = 0.45 #全色感光层吸收的蓝光
        color_type = ("single") #色彩类型
        sens_factor = 1.0 #高光敏感系数
        d_r = 0 #红色感光层接受的散射光
        l_r = 0 #红色感光层接受的直射光
        x_r = 0 #红色感光层的响应系数
        n_r = 0 #红色感光层的颗粒度
        d_g = 0 #绿色感光层接受的散射光
        l_g = 0 #绿色感光层接受的直射光
        x_g = 0 #绿色感光层的响应系数
        n_g = 0 #绿色感光层的颗粒度
        d_b = 0 #蓝色感光层接受的散射光
        l_b = 0 #蓝色感光层接受的直射光
        x_b = 0 #蓝色感光层的响应系数
        n_b = 0 #蓝色感光层的颗粒度
        d_l = 2.33 #全色感光层接受的散射光
        l_l = 0.85 #全色感光层接受的直射光
        x_l = 1.15 #全色感光层的响应系数
        n_l = 0.20 #全色感光层的颗粒度
        gamma = 2.2
        A = 0.15 #肩部强度
        B = 0.50 #线性段强度
        C = 0.10 #线性段平整度
        D = 0.20 #趾部强度
        E = 0.02 #趾部硬度
        F = 0.30 #趾部软度
    elif film_type == ("AS100"):
        r_r = 0 #红色感光层吸收的红光
        r_g = 0 #红色感光层吸收的绿光
        r_b = 0 #红色感光层吸收的蓝光
        g_r = 0 #绿色感光层吸收的红光
        g_g = 0 #绿色感光层吸收的绿光
        g_b = 0 #绿色感光层吸收的蓝光
        b_r = 0 #蓝色感光层吸收的红光
        b_g = 0 #蓝色感光层吸收的绿光
        b_b = 0 #蓝色感光层吸收的蓝光
        t_r = 0.30 #全色感光层吸收的红光
        t_g = 0.12 #全色感光层吸收的绿光
        t_b = 0.45 #全色感光层吸收的蓝光
        color_type = ("single") #色彩类型
        sens_factor = 1.28 #高光敏感系数
        d_r = 0 #红色感光层接受的散射光
        l_r = 0 #红色感光层接受的直射光
        x_r = 0 #红色感光层的响应系数
        n_r = 0 #红色感光层的颗粒度
        d_g = 0 #绿色感光层接受的散射光
        l_g = 0 #绿色感光层接受的直射光
        x_g = 0 #绿色感光层的响应系数
        n_g = 0 #绿色感光层的颗粒度
        d_b = 0 #蓝色感光层接受的散射光
        l_b = 0 #蓝色感光层接受的直射光
        x_b = 0 #蓝色感光层的响应系数
        n_b = 0 #蓝色感光层的颗粒度
        d_l = 1.0 #全色感光层接受的散射光
        l_l = 1.05 #全色感光层接受的直射光
        x_l = 1.25 #全色感光层的响应系数
        n_l = 0.10 #全色感光层的颗粒度
        gamma = 2.0
        A = 0.15 #肩部强度
        B = 0.50 #线性段强度
        C = 0.25 #线性段平整度
        D = 0.35 #趾部强度
        E = 0.02 #趾部硬度
        F = 0.35 #趾部软度
        
    return r_r,r_g,r_b,g_r,g_g,g_b,b_r,b_g,b_b,t_r,t_g,t_b,color_type,sens_factor,d_r,l_r,x_r,n_r,d_g,l_g,x_g,n_g,d_b,l_b,x_b,n_b,d_l,l_l,x_l,n_l,gamma,A,B,C,D,E,F


In [ ]:
%%writefile core/processing.py
import cv2
import time
from .presets import film_choose
from .backend import xp, to_cpu, to_gpu, gaussian_blur, resize

def standardize(image, min_size=3000):
    """标准化图像尺寸"""
    
    # If min_size is <= 0, do not resize (keep original)
    if min_size <= 0:
        return image

    # 获取原始尺寸
    # Ensure image size calculation works on both
    height, width = image.shape[:2]
    
    # If image is already smaller than min_size, maybe don't upscale? 
    # Original logic always resized based on short edge being min_size (so it upscales small images too).
    # We keep original logic for consistency unless min_size is passed.
    
    # 确定缩放比例
    if height < width:
        # 竖图 - 高度为短边
        scale_factor = min_size / height
        new_height = min_size
        new_width = int(width * scale_factor)
    else:
        # 横图 - 宽度为短边
        scale_factor = min_size / width
        new_width = min_size
        new_height = int(height * scale_factor)
    
    # 确保新尺寸为偶数（避免某些处理问题）
    new_width = new_width + 1 if new_width % 2 != 0 else new_width
    new_height = new_height + 1 if new_height % 2 != 0 else new_height
    
    interpolation = cv2.INTER_AREA if scale_factor < 1 else cv2.INTER_LANCZOS4
    
    # Use backend resize
    resized_image = resize(image, (new_width, new_height), interpolation=interpolation)

    return resized_image
    #统一尺寸

# ... (luminance, average, grain, reinhard, filmic, opt unchanged) ...

def process(image, film_type, grain_style, Tone_style, resolution=3000):
    
    start_time = time.time()
    
    # Upload to GPU if available
    image = to_gpu(image)

    # 获取胶片参数
    (r_r,r_g,r_b,g_r,g_g,g_b,b_r,b_g,b_b,t_r,t_g,t_b,color_type,sens_factor,d_r,l_r,x_r,n_r,d_g,l_g,x_g,n_g,d_b,l_b,x_b,n_b,d_l,l_l,x_l,n_l,gamma,A,B,C,D,E,F) = film_choose(film_type)
    
    if grain_style == ("默认"):
        n_r = n_r * 1.0
        n_g = n_g * 1.0
        n_b = n_b * 1.0
        n_l = n_l * 1.0
    elif grain_style == ("柔和"):
        n_r = n_r * 0.5
        n_g = n_g * 0.5
        n_b = n_b * 0.5
        n_l = n_l * 0.5
    elif grain_style == ("较粗"):
        n_r = n_r * 1.5
        n_g = n_g * 1.5
        n_b = n_b * 1.5
        n_l = n_l * 1.5
    elif grain_style == ("不使用"):
        n_r = n_r * 0
        n_g = n_g * 0
        n_b = n_b * 0
        n_l = n_l * 0


    # 调整尺寸
    image = standardize(image, min_size=resolution)

    (lux_r,lux_g,lux_b,lux_total) = luminance(image,color_type,r_r,r_g,r_b,g_r,g_g,g_b,b_r,b_g,b_b,t_r,t_g,t_b)
    #重建光线
    film = opt(lux_r,lux_g,lux_b,lux_total,color_type, sens_factor, d_r, l_r, x_r, n_r, d_g, l_g, x_g, n_g, d_b, l_b, x_b, n_b, d_l, l_l, x_l, n_l,grain_style,gamma,A,B,C,D,E,F,Tone_style)
    #冲洗底片
    
    # Download from GPU to CPU (ensure result is numpy)
    film = to_cpu(film)

    process_time = time.time() - start_time

    return film, process_time
    #统一尺寸

def luminance(image,color_type,r_r,r_g,r_b,g_r,g_g,g_b,b_r,b_g,b_b,t_r,t_g,t_b):
    """计算亮度图像 (0-1范围)"""
    # 分离RGB通道 - Replace cv2.split with slicing
    b = image[:, :, 0]
    g = image[:, :, 1]
    r = image[:, :, 2]
    
    # 转换为浮点数
    b_float = xp.astype(b, xp.float32) / 255.0
    g_float = xp.astype(g, xp.float32) / 255.0
    r_float = xp.astype(r, xp.float32) / 255.0
    
    # 模拟不同乳剂层的吸收特性
    if color_type == ("color"):
        lux_r = r_r * r_float + r_g * g_float + r_b * b_float
        lux_g = g_r * r_float + g_g * g_float + g_b * b_float
        lux_b = b_r * r_float + b_g * g_float + b_b * b_float
        lux_total = t_r * r_float + t_g * g_float + t_b * b_float
    else:
        lux_total = t_r * r_float + t_g * g_float + t_b * b_float
        lux_r = None
        lux_g = None
        lux_b = None

    return lux_r,lux_g,lux_b,lux_total
    #实现对源图像的分光并整合输出

def average(lux_total):
    """计算图像的平均亮度 (0-1)"""
    # 计算平均亮度
    avg_lux = xp.mean(lux_total)
    avg_lux = xp.clip(avg_lux,0,1)
    return avg_lux
    #计算平均亮度

def grain(lux_r,lux_g,lux_b,lux_total,color_type,sens):
    #基于加权随机的颗粒模拟
    if color_type == ("color"):

        # 创建正负噪声
        noise = xp.astype(xp.random.normal(0,1, lux_r.shape), xp.float32)
        noise = noise ** 2
        noise = noise * (xp.random.choice([-1, 1],lux_r.shape))
        # 创建权重图 (中等亮度区域权重最高)
        weights =(0.5 - xp.abs(lux_r - 0.5)) * 2
        weights = xp.clip(weights,0.05,0.9)
        # 应用权重
        sens_grain = xp.clip (sens,0.4,0.6)
        weighted_noise = noise * weights* sens_grain
        # 添加轻微模糊
        # Use backend gaussian_blur
        weighted_noise = gaussian_blur(weighted_noise, (3, 3), 1)
        weighted_noise_r = xp.clip(weighted_noise, -1,1)
        # 应用颗粒

        # 创建正负噪声
        noise = xp.astype(xp.random.normal(0,1, lux_g.shape), xp.float32)
        noise = noise ** 2
        noise = noise * (xp.random.choice([-1, 1],lux_g.shape))
        # 创建权重图 (中等亮度区域权重最高)
        weights =(0.5 - xp.abs(lux_g - 0.5)) * 2
        weights = xp.clip(weights,0.05,0.9)
        # 应用权重
        sens_grain = xp.clip (sens,0.4,0.6)
        weighted_noise = noise * weights* sens_grain
        # 添加轻微模糊
        weighted_noise = gaussian_blur(weighted_noise, (3, 3), 1)
        weighted_noise_g = xp.clip(weighted_noise, -1,1)
        # 应用颗粒

        # 创建正负噪声
        noise = xp.astype(xp.random.normal(0,1, lux_b.shape), xp.float32)
        noise = noise ** 2
        noise = noise * (xp.random.choice([-1, 1],lux_b.shape))
        # 创建权重图 (中等亮度区域权重最高)
        weights =(0.5 - xp.abs(lux_b - 0.5)) * 2
        weights = xp.clip(weights,0.05,0.9)
        # 应用权重
        sens_grain = xp.clip (sens,0.4,0.6)
        weighted_noise = noise * weights* sens_grain
        # 添加轻微模糊
        weighted_noise = gaussian_blur(weighted_noise, (3, 3), 1)
        weighted_noise_b = xp.clip(weighted_noise, -1,1)
        
        weighted_noise_total = None
        # 应用颗粒
        
    else:

        # 创建正负噪声
        noise = xp.astype(xp.random.normal(0,1, lux_total.shape), xp.float32)
        noise = noise ** 2
        noise = noise * (xp.random.choice([-1, 1],lux_total.shape))
        # 创建权重图 (中等亮度区域权重最高)
        weights =(0.5 - xp.abs(lux_total - 0.5)) * 2
        weights = xp.clip(weights,0.05,0.9)
        # 应用权重
        sens_grain = xp.clip (sens,0.4,0.6)
        weighted_noise = noise * weights* sens_grain
        # 添加轻微模糊
        weighted_noise = gaussian_blur(weighted_noise, (3, 3), 1)
        weighted_noise_total = xp.clip(weighted_noise, -1,1)
        weighted_noise_r = None
        weighted_noise_g = None
        weighted_noise_b = None
        # 应用颗粒
    
    return weighted_noise_r,weighted_noise_g,weighted_noise_b,weighted_noise_total
    #创建颗粒函数

def reinhard(lux_r,lux_g,lux_b,lux_total,color_type,gamma):
    #定义reinhard算法，exp为曝光度，gam为伽马值
    
    if color_type == "color":

        mapped = lux_r
        #定义输入的图像
        mapped = mapped * (mapped/ (1.0 + mapped))
        #应用reinhard算法
        mapped = xp.power(mapped, 1.05/gamma)
        result_r = xp.clip(mapped,0,1)

        mapped = lux_g
        #定义输入的图像
        mapped = mapped * (mapped/ (1.0 + mapped))
        #应用reinhard算法
        mapped = xp.power(mapped, 1.05/gamma)
        result_g = xp.clip(mapped,0,1)

        mapped = lux_b
        #定义输入的图像
        mapped = mapped * (mapped/ (1.0 + mapped))
        #应用reinhard算法
        mapped = xp.power(mapped, 1.05/gamma)
        result_b = xp.clip(mapped,0,1)
        result_total = None
    else:
        mapped = lux_total
        #定义输入的图像
        mapped = mapped * (mapped/ (1.0 + mapped))
        #应用reinhard算法
        mapped = xp.power(mapped, 1.0/gamma)
        result_total = xp.clip(mapped,0,1)
        result_r = None
        result_g = None
        result_b = None

    return result_r,result_g,result_b,result_total
    #创建reinhard函数

def filmic(lux_r,lux_g,lux_b,lux_total,color_type,gamma,A,B,C,D,E,F):
    #fimlic映射

    if color_type == ("color"):

        lux_r = xp.maximum(lux_r, 0)
        lux_g = xp.maximum(lux_g, 0)
        lux_b = xp.maximum(lux_b, 0)

        lux_r = 10 * (lux_r ** gamma)
        lux_g = 10 * (lux_g ** gamma)
        lux_b = 10 * (lux_b ** gamma)

        result_r = ((lux_r * (A * lux_r + C * B) + D * E) / (lux_r * (A * lux_r + B) + D * F)) - E/F
        result_g = ((lux_g * (A * lux_g + C * B) + D * E) / (lux_g * (A * lux_g + B) + D * F)) - E/F
        result_b = ((lux_b * (A * lux_b + C * B) + D * E) / (lux_b * (A * lux_b + B) + D * F)) - E/F
        result_total = None
    else:
        lux_total = xp.maximum(lux_total, 0)
        lux_total = 10 * (lux_total ** gamma)
        result_r = None
        result_g = None
        result_b = None
        result_total = ((lux_total * (A * lux_total + C * B) + D * E) / (lux_total * (A * lux_total + B) + D * F)) - E/F
    
    return result_r,result_g,result_b,result_total

def opt(lux_r,lux_g,lux_b,lux_total,color_type, sens_factor, d_r, l_r, x_r, n_r, d_g, l_g, x_g, n_g, d_b, l_b, x_b, n_b, d_l, l_l, x_l, n_l,grain_style,gamma,A,B,C,D,E,F,Tone_style):
    #opt 光学扩散函数

    avrl = average(lux_total)
    # 根据平均亮度计算敏感度
    sens = (1.0 - avrl) * 0.75 + 0.10
    # 将敏感度限制在0-1范围内
    sens = xp.clip(sens,0.10,0.7) #sens -- 高光敏感度
    strg = 23 * sens**2 * sens_factor #strg -- 光晕强度

    base = 0.05 * sens_factor #base -- 基础扩散强度

    # Downsampling factor for bloom optimization
    bloom_scale = 0.125  # 1/8 size

    # ksize not strictly needed for backend blur if we pass sigma, 
    # but we derived it from params so let's check
    # In backend.gaussian_blur we handle sigma.
    # The original logic used `rads` (radius) to determine kernel size for cv2.
    # cv2.GaussianBlur(..., (ksize, ksize), 1) -> sigma=1 (fixed??)
    # Wait, original code:
    # weighted_noise = cv2.GaussianBlur(weighted_noise, (3, 3), 1) # Sigma=1
    # bloom:
    # bloom_layer_small = cv2.GaussianBlur(lux_small * weights_small, (0, 0), sigma_small)
    # where sigma_small = sens * sigma_mult * bloom_scale
    pass
    
    # Pre-calculate scaled dimensions
    h_small = int(lux_total.shape[0] * bloom_scale)
    w_small = int(lux_total.shape[1] * bloom_scale)
    
    # Helper to process bloom channel
    def process_bloom_channel(lux_channel, sens, sigma_mult):
        # Resize down
        lux_small = resize(lux_channel, (w_small, h_small), interpolation=cv2.INTER_LINEAR)
        
        # Calculate weights on small image
        weights_small = (base + lux_small**2) * sens
        weights_small = xp.clip(weights_small, 0, 1)
        
        # Blur on small image
        sigma_small = sens * sigma_mult * bloom_scale
        
        # Use backend blur. (0,0) logic handled by passing sigma directly.
        bloom_layer_small = gaussian_blur(lux_small * weights_small, (0, 0), sigma_small)
        
        # Compute effect on small image
        bloom_effect_small = bloom_layer_small * weights_small * strg
        bloom_effect_small = (bloom_effect_small / (1.0 + bloom_effect_small))
        
        # Resize up
        bloom_effect_large = resize(bloom_effect_small, (lux_channel.shape[1], lux_channel.shape[0]), interpolation=cv2.INTER_LINEAR)
        return bloom_effect_large

    if color_type == ("color"):
        bloom_effect_r = process_bloom_channel(lux_r, sens, 55)
        bloom_effect_g = process_bloom_channel(lux_g, sens, 35)
        bloom_effect_b = process_bloom_channel(lux_b, sens, 15)
        
        if grain_style == ("不使用"):
            lux_r = bloom_effect_r * d_r + (lux_r**x_r) * l_r
            lux_g = bloom_effect_g * d_g + (lux_g**x_g) * l_g
            lux_b = bloom_effect_b * d_b + (lux_b**x_b) * l_b
        else:    
            (weighted_noise_r,weighted_noise_g,weighted_noise_b,weighted_noise_total) = grain(lux_r,lux_g,lux_b,lux_total,color_type,sens)
            #应用颗粒
            lux_r = bloom_effect_r * d_r + (lux_r**x_r) * l_r + weighted_noise_r *n_r + weighted_noise_g *n_l+ weighted_noise_b *n_l
            lux_g = bloom_effect_g * d_g + (lux_g**x_g) * l_g + weighted_noise_r *n_l + weighted_noise_g *n_g+ weighted_noise_b *n_l
            lux_b = bloom_effect_b * d_b + (lux_b**x_b) * l_b + weighted_noise_r *n_l + weighted_noise_g *n_l + weighted_noise_b *n_b
        
        #拼合光层
        if Tone_style == "filmic":
            (result_r,result_g,result_b,result_total) = filmic(lux_r,lux_g,lux_b,lux_total,color_type,gamma,A,B,C,D,E,F)
            #应用flimic映射
        else:
            (result_r,result_g,result_b,result_total) = reinhard(lux_r,lux_g,lux_b,lux_total,color_type,gamma)
            #应用映射

        combined_b = xp.astype((result_b * 255), xp.uint8)
        combined_g = xp.astype((result_g * 255), xp.uint8)
        combined_r = xp.astype((result_r * 255), xp.uint8)
        # Use xp.dstack (depth stack) for merging
        film = xp.dstack([combined_r, combined_g, combined_b])
    else:
        bloom_effect = process_bloom_channel(lux_total, sens, 55)

        #应用光晕
        if grain_style == ("不使用"):
            lux_total = bloom_effect * d_l + (lux_total**x_l) * l_l
        else:
            (weighted_noise_r,weighted_noise_g,weighted_noise_b,weighted_noise_total) = grain(lux_r,lux_g,lux_b,lux_total,color_type,sens)
            #应用颗粒
            lux_total = bloom_effect * d_l + (lux_total**x_l) * l_l + weighted_noise_total *n_l
        
        #拼合光层
        
        if Tone_style == "filmic":
            (result_r,result_g,result_b,result_total) = filmic(lux_r,lux_g,lux_b,lux_total,color_type,gamma,A,B,C,D,E,F)
            #应用flimic映射
        else:
            (result_r,result_g,result_b,result_total) = reinhard(lux_r,lux_g,lux_b,lux_total,color_type,gamma)
            #应用reinhard映射

        film = xp.astype((result_total * 255), xp.uint8)

    return film
    #返回渲染后的光度
    #进行底片成像
    #准备暗房工具




In [ ]:
#@title 3. Upload File (Image or Video)

uploaded = files.upload()
input_filename = next(iter(uploaded))
ext = os.path.splitext(input_filename)[1].lower()

if ext in ['.jpg', '.jpeg', '.png', '.bmp']:
    display(Image(input_filename, width=300))
    file_mode = "image"
elif ext in ['.mp4', '.avi', '.mov', '.mkv', '.m4v', '.webm', '.flv']:
    print(f"Video file uploaded: {input_filename}")
    file_mode = "video"
else:
    print("Unknown file type, converting as image...")
    file_mode = "image"

print(f"Uploaded: {input_filename} (Mode: {file_mode})")

In [ ]:
#@title 4. Process
#@markdown Select parameters. For video, resolution=0 keeps original size (recommended for video).

film_type = "NC200" #@param ["NC200", "AS100", "FS200"]
grain_style = "\u9ED8\u8BA4" #@param ["\u9ED8\u8BA4", "\u67D4\u548C", "\u8F83\u7C97", "\u4E0D\u4F7F\u7528"]
tone_style = "reinhard" #@param ["filmic", "reinhard"]
target_resolution = 0 #@param {type:"integer", help:"Short edge length. Set 0 to keep original size (recommended for video). Default 3000 for high-res photos."}

from core.processing import process
from core.backend import get_backend_name

def process_and_display_image(filename):
    img = cv2.imread(filename)
    if img is None:
        print("Error loading image")
        return
        
    print(f"Processing Image with {get_backend_name()}...")
    res = target_resolution
    if res <= 0:
        res = -1 # Skip resize
    else:
        print(f"Target resolution: {res} (short edge)")
        
    film, p_time = process(img, film_type, grain_style, tone_style, resolution=res)
    
    print(f"Done in {p_time:.2f}s")
    
    output_filename = f"processed_{film_type}_{int(time.time())}.jpg"
    film_bgr = cv2.cvtColor(film, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_filename, film_bgr)
    
    display(Image(output_filename, width=500))
    return output_filename

def process_video(filename):
    cap = cv2.VideoCapture(filename)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Processing Video: {width}x{height} @ {fps}fps ({total_frames} frames)")
    print(f"Device: {get_backend_name()}")
    
    res = target_resolution
    if res <= 0: 
        res = -1 # Keep original
    else:
        # Calculate target dimensions for writer
        if height < width:
            scale = res / height
            new_h = res
            new_w = int(width * scale)
        else:
            scale = res / width
            new_w = res
            new_h = int(height * scale)
        width, height = new_w + (new_w%2), new_h + (new_h%2)
        
    timestamp = int(time.time())
    base_name = f"processed_{film_type}_{timestamp}"
    final_output_filename = f"{base_name}.mp4"
    mute_filename = f"{base_name}_mute.mp4"
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(mute_filename, fourcc, fps, (width, height))
    
    try:
        with tqdm(total=total_frames, unit='frame') as pbar:
            while True:
                ret, frame = cap.read()
                if not ret: break
                
                film_rgb, _ = process(frame, film_type, grain_style, tone_style, resolution=res)
                film_bgr = cv2.cvtColor(film_rgb, cv2.COLOR_RGB2BGR)
                out.write(film_bgr)
                pbar.update(1)
    finally:
        cap.release()
        out.release()
        
    # Check for ffmpeg and merge audio
    if shutil.which('ffmpeg') is not None:
        print("Merging audio...")
        cmd = [
            'ffmpeg', '-y',
            '-i', mute_filename,
            '-i', filename,
            '-c:v', 'copy',
            '-c:a', 'aac',
            '-map', '0:v:0',
            '-map', '1:a:0',
            '-shortest',
            final_output_filename
        ]
        try:
            subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
            print(f"Audio merged. Saved to {final_output_filename}")
            os.remove(mute_filename)
            return final_output_filename
        except subprocess.CalledProcessError as e:
             print(f"Audio merge failed. Returning mute video. Error: {e}")
             # Rename mute to final so user gets the expected file
             if os.path.exists(mute_filename):
                 os.rename(mute_filename, final_output_filename)
             return final_output_filename
    else:
         print("ffmpeg not found. Returning mute video.")
         if os.path.exists(mute_filename):
             os.rename(mute_filename, final_output_filename)
         return final_output_filename

if file_mode == "image":
    output_file = process_and_display_image(input_filename)
else:
    output_file = process_video(input_filename)


In [ ]:
#@title 5. Download Result
if 'output_file' in locals():
    files.download(output_file)
else:
    print("No output file generated yet.")